# Import Library dan Insisialisasi API

In [9]:
import googleapiclient.discovery # library untuk akses API Google
import pandas as pd

# Inisialisasi API
api_service_name = "youtube" # jenis layanan API
api_version = "v3" # Versi API
DEVELOPER_KEY = "Your API YouTube" # API key dari Google cloud console

# membangung akses endpoint dari YouTube API
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY)

# Proses Pengambilan Comments

In [10]:
# Inisialisasi variabel
video_id = "9s6imDGjy08" # ID video youtube yang ingin di scrape
comments = [] # list utk menyimpan comment
next_page_token = None # token untuk pagination
total_comments = 0 # hitung jumlah comment yg diambil
max_comments = 10000  # Jumlah target komentar

# Looping utk pengambilan comments
while total_comments < max_comments:
    # Kirim request ke API YouTube
    request = youtube.commentThreads().list(
        part="snippet",
        videoId=video_id,
        maxResults=100, # jumlah comment maksimal yg bisa diambil dari api yt
        pageToken=next_page_token # mengambil halaman berikutnya dari comment
    )
    response = request.execute()

    # Ambil Data Komentar dari Hasil Response
    for item in response['items']:
        comment = item['snippet']['topLevelComment']['snippet']
        # menambahkan data ke comments
        comments.append([
            comment['authorDisplayName'], # Nama pengguna yang berkomentar
            comment['publishedAt'], # Tanggal komentar dibuat
            comment['updatedAt'], # Tanggal komentar terakhir diubah
            comment['likeCount'], # Jumlah likes pada komentar
            comment['textDisplay'] # Isi komentar
        ])
        total_comments += 1

        if total_comments >= max_comments:
            break
        
    # Pindah ke Halaman Selanjutnya        
    next_page_token = response.get('nextPageToken')
    if not next_page_token:
        break  # Tidak ada halaman lagi


# Analisis Hasil Scraping

In [15]:
# Simpan ke DataFrame
df = pd.DataFrame(comments, columns=['author', 'published_at', 'updated_at', 'like_count', 'text'])

# Tampilkan 10 komentar pertama, info dataset, dan shapenya
print(df.head(10),'\n')
print('\n',df.info())
print('\n',df.shape)

             author          published_at            updated_at  like_count  \
0      @marvell4636  2025-03-24T13:52:03Z  2025-03-24T13:52:03Z           0   
1    @TRICKSTER-y3n  2025-03-24T12:10:52Z  2025-03-24T12:10:52Z           0   
2        @Luckyyy_.  2025-03-24T10:53:52Z  2025-03-24T10:53:52Z           2   
3   @andigunaya6145  2025-03-24T07:26:22Z  2025-03-24T07:26:22Z           0   
4       @ffoldyeard  2025-03-24T03:36:01Z  2025-03-24T03:36:01Z           1   
5         @aby_0280  2025-03-24T02:50:55Z  2025-03-24T02:50:55Z           1   
6        @abid17329  2025-03-23T15:49:22Z  2025-03-23T15:49:22Z           0   
7  @hartoyovivo3372  2025-03-23T12:45:32Z  2025-03-23T12:45:32Z           0   
8  @Rizkyamelalgara  2025-03-23T12:20:24Z  2025-03-23T12:20:24Z           0   
9      @Akaspribadi  2025-03-23T10:59:26Z  2025-03-23T10:59:26Z           0   

                                                text  
0  <a href="https://www.youtube.com/watch?v=9s6im...  
1                   

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   author        10000 non-null  object
 1   published_at  10000 non-null  object
 2   updated_at    10000 non-null  object
 3   like_count    10000 non-null  int64 
 4   text          10000 non-null  object
dtypes: int64(1), object(4)
memory usage: 390.8+ KB


In [ ]:
# cek comment dengan jumlah like terbanyak
df.groupby(by=['author','text']).agg({
    'like_count': 'max'
}).sort_values(by='like_count', ascending=False)

,,like_count
author,text,
@MedyRenaldy,Pls take care of my mini Ronaldo. And dont forget to kiss it every night,1776
@Nerunvlogs,If this comment gets Likes that means Everyone in 2025 is still watching 😮,1416
@Tukangmainzz,"<a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=323"">5:23</a> bro got stream sniped really fast",867
@L4R0n,"IshowSpeed Moments in Indonesia\r<br>\r<br><a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=322"">5:22</a> - Fastest Stream Sniper\r<br><a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=896"">14:56</a> - Speed got his first batik in indonesia\r<br><a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=992"">16:32</a> - Fortune Teller\r<br><a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=1152"">19:12</a> - IShowSpeed United and got a Manchester United batik\r<br><a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=1250"">20:50</a> - Indonesian soldier living statue\r<br><a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=1368"">22:48</a> - Speed tries gorengan (fritter)\r<br><a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=1460"">24:20</a> - Speed tries nasi Padang\r<br><a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=1598"">26:38</a> - Indonesian traditional music performer\r<br><a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=1666"">27:46</a> - Speed playing angklung (traditional musical instrument)\r<br><a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=1878"">31:18</a> - Another Indonesian soldier living statue\r<br><a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=2000"">33:20</a> - Speed sings a duet with a female\r<br><a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=2546"">42:26</a> - Speed tries Indonesian KFC\r<br><a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=2922"">48:42</a> - Speed in a horror house\r<br><a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=3010"">50:10</a> - Speed helping a zombie out of prison\r<br><a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=3226"">53:46</a> - Speed helping a crawling ghost.\r<br><a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=3283"">54:43</a> - Speed bodyguard as a ghost, W henry\r<br><a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=3505"">58:25</a> - Crowd\r<br><a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=5095"">1:24:55</a> - Speed hits 1 million live views \r<br><a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=9610"">2:40:10</a> - W Fan Art\r<br><a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=9810"">2:43:30</a> - 20 vs 1\r<br><a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=10713"">2:58:33</a> - Speed vs his craziest fan\r<br><a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=11100"">3:05:00</a> - Speed gets a Cristiano Ronaldo detailed Hot Toys gift\r<br><a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=11595"">3:13:15</a> - Speed and a wannabe monkey\r<br><a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=11681"">3:14:41</a> - Speed vs Messi fans\r<br><a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=12229"">3:23:49</a> - 20 vs 1 and Nami\r<br><a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=13616"">3:46:56</a> - Green Apple",526
@Saugat.Timalsina,Who&#39;s here after PERU stream <br>👇,524
...,...,...
@SujiwoTantri,luaaaaarrrr biasaaaaaaaaa dilasi jackpot teruss sama𝗞𝗗𝘀𝗹𝗼𝘁𝘀 🍒,0
@SuipOP00,"<a href=""https://www.youtube.com/watch?v=9s6imDGjy08&amp;t=11123"">3:05:23</a> bro was crying for his life",0
@SuhartiniSuhartini-o2t,minggir lu miskin,0


# Simpan Dataset dalam Format CSV

In [17]:
# Simpan ke CSV
df.to_csv("IShowSpeed_in_IDN.csv", index=False)